In [ ]:
# importing ARP dataset 
import pandas as pd
df=pd.read_csv(r"C:\Users\DELL\Desktop\MINOR PROJECT\ARP Poison,flood attack Dataset.csv")
df

In [ ]:
# Droping duplicates rows
df.drop_duplicates(inplace=True)

In [ ]:
# Droping rows containg null value
df.dropna(inplace=True)
df.isnull().sum()

In [ ]:
for i in df.columns:
  print(i,len(df[i].unique()))

In [ ]:
# preprocessing of data by LabelEncoder

from sklearn import preprocessing

label_encoder = preprocessing.LabelEncoder()

columns=["outport","src_mac_addr(eth)",	"src_mac_addr(arp)","dst_mac_addr(eth)","dst_mac_addr(arp)","src_ip(arp)","dst_ip(arp)","total_time"]
for attr in columns:
    df[attr]= label_encoder.fit_transform(df[attr])

df

In [ ]:
# Load and preprocess data
x=df.iloc[:,:len(df.columns)-1]
X=x
y=df.iloc[:,len(df.columns)-1]


In [ ]:
# merging target attribute to make it binary classification
y.value_counts()


In [ ]:

# y=pd.get_dummies(y)
# y.replace(to_replace=True,value=1,inplace=True)
# y.replace(to_replace=False,value=0,inplace=True)
# y


In [ ]:
# ANN model
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.optimizers import Adam
# from sklearn.metrics import accuracy_score


y=np.array(y)


X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=42)

# print(y_train)
# print(y_test)



sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)




# Initialize and train the ANN classifier
ann= Sequential([

Dense(units=15,activation="relu",input_shape=(15,)),

Dropout(0.5),
Dense(units=32,activation="relu"),

Dropout(0.5),
Dense(units=16,activation="relu"),

Dense(3,activation="softmax")

])


ann.compile(optimizer=Adam(learning_rate=0.000001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])



early_stop=tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0.0001,
    patience=20,
    verbose=1,
    mode="auto",
    baseline=None,
    restore_best_weights=False,
    start_from_epoch=0,
)

model_history=ann.fit(X_train,y_train,validation_split=0.33,epochs=100,batch_size=32,callbacks=early_stop)  





In [ ]:
model_history.history.keys()

In [ ]:
import matplotlib.pyplot as plt

plt.plot(model_history.history['accuracy'])
plt.plot(model_history.history['val_accuracy'])
plt.title('model accuracy')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.legend(['train','test'],loc='upper left')
plt.show()

In [ ]:
import matplotlib.pyplot as plt

plt.plot(model_history.history['loss'])
plt.plot(model_history.history['val_loss'])
plt.title('model loss')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['train','test'],loc='upper left')
plt.show()

In [ ]:
# Predictions
y_pred = np.argmax(ann.predict(X_test), axis=-1)
y_pred

In [ ]:
y_test.shape

In [ ]:
y_pred.shape

In [ ]:

# ANN 
        
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

accuracy=accuracy_score(y_test,y_pred)
print("Accuracy:",accuracy)

print("\nClassification Report:\n",classification_report(y_test,y_pred))
cm=confusion_matrix(y_test,y_pred)
print("\nConfusion Matrix:\n",cm)


In [ ]:
# ANN 

import seaborn as sns
import matplotlib.pyplot as plt

labels=["0","1","2"]
sns.heatmap(cm,annot=True,fmt="d", xticklabels=labels, yticklabels=labels)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()   

In [ ]:
from sklearn.metrics import roc_curve, auc, precision_recall_curve, matthews_corrcoef
import matplotlib.pyplot as plt

# ROC curve

plt.figure(figsize=(8, 6))
for i in range(len(np.unique(y_test))):
    y_prob = ann.predict(X_test)
    fpr, tpr, _ = roc_curve((y_test == i).astype(int), y_prob[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f'Class {i} (AUC = {roc_auc:.2f})')

plt.title('ROC Curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
plt.grid()
plt.show()


# Precision-Recall curve

plt.figure(figsize=(8, 6))
for i in range(len(np.unique(y_test))):
    y_prob = ann.predict(X_test)
    precision, recall, _ = precision_recall_curve((y_test == i).astype(int), y_prob[:, i])
    plt.plot(recall, precision, label=f'Class {i}')

plt.title('Precision-Recall Curve')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.legend()
plt.grid()
plt.show()


# Matthews correlation coefficient (MCC)
mcc = matthews_corrcoef(y_test, y_pred)

print("Matthews Correlation Coefficient :", mcc)
